In [1]:
import os
import csv
import sys
import random
import scipy
import numpy as np
from itertools import izip


model_no = 19

workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

label_dict = dict()

label_dict[1] = 'ind_ahor_fin_ult1'
label_dict[2] = 'ind_aval_fin_ult1'
label_dict[3] = 'ind_cco_fin_ult1'
label_dict[4] = 'ind_cder_fin_ult1'
label_dict[5] = 'ind_cno_fin_ult1'
label_dict[6] = 'ind_ctju_fin_ult1'
label_dict[7] = 'ind_ctma_fin_ult1'
label_dict[8] = 'ind_ctop_fin_ult1'
label_dict[9] = 'ind_ctpp_fin_ult1'
label_dict[10] = 'ind_deco_fin_ult1'
label_dict[11] = 'ind_deme_fin_ult1'
label_dict[12] = 'ind_dela_fin_ult1'
label_dict[13] = 'ind_ecue_fin_ult1'
label_dict[14] = 'ind_fond_fin_ult1'
label_dict[15] = 'ind_hip_fin_ult1'
label_dict[16] = 'ind_plan_fin_ult1'
label_dict[17] = 'ind_pres_fin_ult1'
label_dict[18] = 'ind_reca_fin_ult1'
label_dict[19] = 'ind_tjcr_fin_ult1'
label_dict[20] = 'ind_valo_fin_ult1'
label_dict[21] = 'ind_viv_fin_ult1'
label_dict[22] = 'ind_nomina_ult1'
label_dict[23] = 'ind_nom_pens_ult1'
label_dict[24] = 'ind_recibo_ult1'


In [3]:
#train_read = open(workspace_path + '/train/model_pf02/cust_product_'+str(model_no)+'.train', 'r')
#Ein_read = open(workspace_path + '../liblinear-2.1/output/cust_product_'+str(model_no)+'_inv.Ein', 'r')

model_no = 19

for model_no in range(1, 25):

    test_data_path = './test/cust_product_inv.test'
    model_path = '../liblinear-2.1/model_save/cust_product_'+str(model_no)+'_inv.model'
    Eout_path = '../liblinear-2.1/output/cust_product_'+str(model_no)+'_inv.Eout'

    
    predi_command = '../liblinear-2.1/./predict'
    #print os.system("pwd");
    os.system(predi_command +' '+ test_data_path +' '+ model_path +' '+ Eout_path);



In [ ]:
Eout_results_id_2_prod = dict()




for model_no in range(1, 25):

    feat_data_path = './feat/2_cust_product_inv.feat'
    Eout_path = '../liblinear-2.1/output/cust_product_'+str(model_no)+'_inv.Eout'

    test_read = open(feat_data_path, 'r')
    Eout_read = open(Eout_path, 'r')
    
    check_line_count = 0
    
    with test_read as test_data, Eout_read as predict_label: 
         for line1, line2 in izip(test_data, predict_label):
             line1 = line1.strip()
             line2 = line2.strip()
    
             check_line_count += 1
    
             #get the cust_id
             cust_id = ''   
             if len(line1.split('|')) != 2:
                print 'error_line:'+line1
             else:
                cust_id = line1.split('|')[0]
                
             if cust_id not in Eout_results_id_2_prod:
                Eout_results_id_2_prod[cust_id] = ''    
               
             if line2 == '1':
                Eout_results_id_2_prod[cust_id] += label_dict[model_no] + ' '  
                
                 
    test_read.close()
    Eout_read.close()

    print check_line_count  
    
Eout_write = open('./output/Cust2Prod_pred_Nov16.eout', 'w')  

for item in Eout_results_id_2_prod:
    
    prt_line = item + '|' + Eout_results_id_2_prod[item][:-1] + '\n'
    
    Eout_write.write(prt_line)
   
Eout_write.close()

In [ ]:
feat_read = open(workspace_path + '/feat/2_cust_product_inv.feat', 'r')

cust_end_products_dict = dict()

check_line_count = 0

for line in feat_read:
    line = line.strip()
    
#for i in range(0,10000):    
    #line = feat_read.readline().strip()
    
    check_line_count += 1
    
    cust_id = ''
    feat_items = ''
    
    if len(line.split('|')) != 2:
       print 'error_line:'+line
        
    else:
    
       cust_id = line.split('|')[0]
       feat_items = line.split('|')[1]
    
    if cust_id not in cust_end_products_dict:
       cust_end_products_dict[cust_id] = [] 
    
    if len(feat_items) > 1: 
        
        if len(feat_items.split('$')) != 2:
           print line
    
        label_tokens_get = feat_items.split('$')[0]
        
        for feat_item in label_tokens_get.split(' '):
            
            prod_id = int(feat_item.split(':')[0])
            
            if prod_id < 25:
               cust_end_products_dict[cust_id].append(label_dict[prod_id])
        
print check_line_count        

In [9]:
print cust_end_products_dict['15998']

['ind_nom_pens_ult1', 'ind_nomina_ult1']


In [20]:
Eout_dict_read = open('./output/Cust2Prod_pred_Nov16.eout', 'r')  
id_2_prod_dict = dict()

for line in Eout_dict_read:
    line = line.strip()
    
    if len(line.split('|')) != 2:
       print 'error_line:'+line
    else:
       cust_id  = line.split('|')[0]
       products = line.split('|')[1]
        
       id_2_prod_dict[cust_id] = products 

    
sample_submission_read = open('./output/sample_submission.csv', 'r')

head_line = 'ncodpers,added_products' + '\n'
submission_write = open('./output/submission_Nov18.csv', 'w')  
submission_write.write(head_line)

check_line_count = 0

duplicated_items_count = 0
#for i in range(0,10000):    
    #line = sample_submission_read.readline().strip()
    
for line in sample_submission_read:
    line = line.strip()    
    
    if 'ncodpers' not in line:
    
        cust_id  = line.split(',')[0]

        prt_line = cust_id + ','
        
        pruned_line = False

        if cust_id in id_2_prod_dict:
            
           
           #print cust_end_products_dict[cust_id]
           #print 'predict:'+ str(id_2_prod_dict[cust_id])  
           
           for prod_name_item in id_2_prod_dict[cust_id].split(' '):
            
               if prod_name_item not in cust_end_products_dict[cust_id]:

                  prt_line += prod_name_item + ' '
                
                  pruned_line = True
                
               else: 
                 
                 #print '->' + prod_name_item    
                 
                 duplicated_items_count += 1

        check_line_count += 1
        
        if pruned_line:
           submission_write.write(prt_line[:-1]+'\n')
        else:
           submission_write.write(prt_line+'\n')  

print check_line_count 
print duplicated_items_count
    
submission_write.close() 
sample_submission_read.close() 
Eout_dict_read.close() 

929615
13845


TP = 8728
FP = 4856
TN = 224378
FN = 2571
precison = 0.64
recall = 0.77
accuracy = 0.97
